In [17]:
import pandas as pd
import numpy as np

# Import the first CSV file
feature_df = pd.read_csv('ufc_features.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Drop the columns
columns_to_drop = ['p1_fighter', 'p2_fighter', 'event_date'] #method
feature_df = feature_df.drop(columns=columns_to_drop)
cols_to_drop = [col for col in feature_df.columns if col.startswith('method_')]
feature_df.drop(columns=cols_to_drop, inplace=True)


# Clean all column names
def clean_column_name(col):
  return col.lower().replace(' ', '_').replace('.', '').replace('-', '_')

# Apply to all columns
feature_df.columns = [clean_column_name(col) for col in feature_df.columns]

# # encode the referee using frequency
# ref_counts = feature_df['referee'].value_counts()
# feature_df['referee_freq'] = feature_df['referee'].map(ref_counts)
# feature_df.drop(columns=['referee'], inplace=True)

feature_df.head()

,referee,winner,p1_height,p1_weight,p1_reach,p1_stance,p1_slpm,p1_str_acc,p1_sapm,p1_str_def,p1_td_avg,p1_td_acc,p1_td_def,p1_sub_avg,p2_height,p2_weight,p2_reach,p2_stance,p2_slpm,p2_str_acc,p2_sapm,p2_str_def,p2_td_avg,p2_td_acc,p2_td_def,p2_sub_avg,p1_age_at_event,p2_age_at_event,height_diff,reach_diff,weight_diff,age_diff,slpm_diff,stracc_diff,sapm_diff,strdef_diff,tdavg_diff,tdacc_diff,tddef_diff,subavg_diff,p1_days_since_last_fight,p2_days_since_last_fight,days_since_last_fight_diff,p1_wins,p1_losses,p1_total,p2_wins,p2_losses,p2_total,win_diff,loss_diff,total_diff,p1_win_streak,p2_win_streak,p1_age_adjusted_slpm,p2_age_adjusted_slpm,p1_age_adjusted_str_acc,p2_age_adjusted_str_acc,p1_age_adjusted_sapm,p2_age_adjusted_sapm,p1_age_adjusted_str_def,p2_age_adjusted_str_def,p1_age_adjusted_td_avg,p2_age_adjusted_td_avg,p1_age_adjusted_td_acc,p2_age_adjusted_td_acc,p1_age_adjusted_td_def,p2_age_adjusted_td_def,p1_age_adjusted_sub_avg,p2_age_adjusted_sub_avg,p1_kd_ema,p2_kd_ema,p1_sig_str_pct_ema,p2_sig_str_pct_ema,p1_td_pct_ema,p2_td_pct_ema,p1_sub_att_ema,p2_sub_att_ema,p1_rev_ema,p2_rev_ema,p1_ctrl_ema,p2_ctrl_ema,p1_r1_kd_ema,p2_r1_kd_ema,p1_r1_sig_str_pct_ema,p2_r1_sig_str_pct_ema,p1_r1_td_pct_ema,p2_r1_td_pct_ema,p1_r1_sub_att_ema,p2_r1_sub_att_ema,p1_r1_rev_ema,p2_r1_rev_ema,p1_r1_ctrl_ema,p2_r1_ctrl_ema,p1_sig_str_pct_detailed_ema,p2_sig_str_pct_detailed_ema,p1_r1_sig_str_pct_detailed_ema,p2_r1_sig_str_pct_detailed_ema,p1_sig_str_landed_ema,p2_sig_str_landed_ema,p1_sig_str_attempted_ema,p2_sig_str_attempted_ema,p1_total_str_landed_ema,p2_total_str_landed_ema,p1_total_str_attempted_ema,p2_total_str_attempted_ema,p1_td_landed_ema,p2_td_landed_ema,p1_td_attempted_ema,p2_td_attempted_ema,p1_r1_sig_str_landed_ema,p2_r1_sig_str_landed_ema,p1_r1_sig_str_attempted_ema,p2_r1_sig_str_attempted_ema,p1_r1_total_str_landed_ema,p2_r1_total_str_landed_ema,p1_r1_total_str_attempted_ema,p2_r1_total_str_attempted_ema,p1_r1_td_landed_ema,p2_r1_td_landed_ema,p1_r1_td_attempted_ema,p2_r1_td_attempted_ema,p1_head_landed_ema,p2_head_landed_ema,p1_head_attempted_ema,p2_head_attempted_ema,p1_body_landed_ema,p2_body_landed_ema,p1_body_attempted_ema,p2_body_attempted_ema,p1_leg_landed_ema,p2_leg_landed_ema,p1_leg_attempted_ema,p2_leg_attempted_ema,p1_distance_landed_ema,p2_distance_landed_ema,p1_distance_attempted_ema,p2_distance_attempted_ema,p1_clinch_landed_ema,p2_clinch_landed_ema,p1_clinch_attempted_ema,p2_clinch_attempted_ema,p1_ground_landed_ema,p2_ground_landed_ema,p1_ground_attempted_ema,p2_ground_attempted_ema,p1_r1_head_landed_ema,p2_r1_head_landed_ema,p1_r1_head_attempted_ema,p2_r1_head_attempted_ema,p1_r1_body_landed_ema,p2_r1_body_landed_ema,p1_r1_body_attempted_ema,p2_r1_body_attempted_ema,p1_r1_leg_landed_ema,p2_r1_leg_landed_ema,p1_r1_leg_attempted_ema,p2_r1_leg_attempted_ema,p1_r1_distance_landed_ema,p2_r1_distance_landed_ema,p1_r1_distance_attempted_ema,p2_r1_distance_attempted_ema,p1_r1_clinch_landed_ema,p2_r1_clinch_landed_ema,p1_r1_clinch_attempted_ema,p2_r1_clinch_attempted_ema,p1_r1_ground_landed_ema,p2_r1_ground_landed_ema,p1_r1_ground_attempted_ema,p2_r1_ground_attempted_ema
0,Dan Miragliotta,1,71.0,135.0,70.0,Switch,5.02,0.44,3.46,0.56,1.19,0.32,0.63,0.3,65.0,135.0,68.0,Orthodox,2.91,0.54,3.55,0.49,1.62,0.34,0.60,1.4,33.034908,37.374401,6.0,2.0,0.0,-4.339493,2.11,-0.10,-0.09,0.07,-0.43,-0.02,0.03,-1.1,273.0,161.0,112.0,10,4,14,13,5,18,-3,-1,-4,0,0,0.151960,0.077861,0.013319,0.014448,0.104738,0.094985,0.016952,0.013111,0.036023,0.043345,0.009687,0.009097,0.019071,0.016054,0.009081,0.037459,0.012025,0.299520,0.427808,0.499030,0.295141,0.250166,0.266130,0.319192,0.032290,0.000000,369.136117,174.851985,0.008057,0.001953,0.434544,0.846666,0.664548,0.467068,0.016114,0.037895,0.000000,0.000000,62.763841,98.073334,0.427808,0.499030,0.434544,0.846666,73.462980,44.916534,168.352377,94.296907,112.881890,50.330579,222.104682,99.836223,2.220167,1.953972,5.331929,8.821761,17.268388,5.361112,38.845877,7.721747,24.855216,8.417673,50.87

In [18]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# --- Encode categorical features ---
# Encode stances
stance_le = LabelEncoder()
all_stances = pd.concat([feature_df['p1_stance'], feature_df['p2_stance']])
stance_le.fit(all_stances)
feature_df['p1_stance'] = stance_le.transform(feature_df['p1_stance'])
feature_df['p2_stance'] = stance_le.transform(feature_df['p2_stance'])

# Encode referee (new additional categorical feature)
referee_le = LabelEncoder()
referee_le.fit(feature_df['referee'])
feature_df['referee'] = referee_le.transform(feature_df['referee'])

# --- Scale numeric features except categorical features ---
numeric_feats = [
    'height', 'weight', 'reach', 'slpm', 'str_acc', 'sapm', 'str_def',
    'td_avg', 'td_acc', 'td_def', 'sub_avg', 'age_at_event',
    # Adding fight history metrics
    # 'days_since_last_fight', 'wins', 'losses', 'total', 'win_streak',
    # Adding age-adjusted metrics
    'age_adjusted_slpm', 'age_adjusted_str_acc', 'age_adjusted_sapm',
    'age_adjusted_str_def', 'age_adjusted_td_avg', 'age_adjusted_td_acc',
    'age_adjusted_td_def', 'age_adjusted_sub_avg',
    # Adding EMA metrics
    'kd_ema', 'sig_str_pct_ema', 'td_pct_ema', 'sub_att_ema', 'rev_ema',
    'ctrl_ema', 'r1_kd_ema', 'r1_sig_str_pct_ema', 'r1_td_pct_ema',
    'r1_sub_att_ema', 'r1_rev_ema', 'r1_ctrl_ema',
    'sig_str_pct_detailed_ema', 'r1_sig_str_pct_detailed_ema',
    'sig_str_landed_ema', 'sig_str_attempted_ema',
    'total_str_landed_ema', 'total_str_attempted_ema',
    'td_landed_ema', 'td_attempted_ema',
    'r1_sig_str_landed_ema', 'r1_sig_str_attempted_ema',
    'r1_total_str_landed_ema', 'r1_total_str_attempted_ema',
    'r1_td_landed_ema', 'r1_td_attempted_ema',
    'head_landed_ema', 'head_attempted_ema',
    'body_landed_ema', 'body_attempted_ema',
    'leg_landed_ema', 'leg_attempted_ema',
    'distance_landed_ema', 'distance_attempted_ema',
    'clinch_landed_ema', 'clinch_attempted_ema',
    'ground_landed_ema', 'ground_attempted_ema',
    'r1_head_landed_ema', 'r1_head_attempted_ema',
    'r1_body_landed_ema', 'r1_body_attempted_ema',
    'r1_leg_landed_ema', 'r1_leg_attempted_ema',
    'r1_distance_landed_ema', 'r1_distance_attempted_ema',
    'r1_clinch_landed_ema', 'r1_clinch_attempted_ema',
    'r1_ground_landed_ema', 'r1_ground_attempted_ema'
]

p1_numeric = ['p1_' + f for f in numeric_feats]
p2_numeric = ['p2_' + f for f in numeric_feats]

# Handle potential NaN values
feature_df[p1_numeric + p2_numeric] = feature_df[p1_numeric + p2_numeric].fillna(0)

scaler = StandardScaler()
feature_df[p1_numeric + p2_numeric] = scaler.fit_transform(feature_df[p1_numeric + p2_numeric])

# --- Prepare features per fighter (including stance) ---
features_per_fighter = numeric_feats + ['stance']
p1_features = ['p1_' + f for f in features_per_fighter]
p2_features = ['p2_' + f for f in features_per_fighter]

# --- Structure data for BiLSTM including referee as a global feature ---
# Create a sequence of [referee, p1_features] and [referee, p2_features] for each sample
X = np.stack([
    np.hstack([feature_df[['referee']].values, feature_df[p1_features].values]),
    np.hstack([feature_df[['referee']].values, feature_df[p2_features].values])
], axis=1)  # shape: (num_samples, 2, 1+num_features_per_fighter)

y = feature_df['winner'].values

# --- Check the shapes and a sample ---
print("X shape:", X.shape)
print("Sample X[0]:\n", X[0])
print("y shape:", y.shape)
print("Sample y:", y[0])
print("Features per fighter (excluding referee):", features_per_fighter)
print("Total features per fighter (including referee):", len(features_per_fighter) + 1)


X shape: (8176, 2, 72)
Sample X[0]:
 [[ 3.50000000e+01  1.88649658e-01 -9.03898155e-01  9.27657219e-02
   1.18725739e+00 -5.41339616e-02  1.04903771e-01  1.95423076e-01
  -3.08185356e-01 -3.49743081e-01  1.16229940e-01 -4.40363611e-01
   5.41652436e-01  7.57168057e-01 -4.13781079e-01 -1.43817431e-01
  -2.73728966e-01 -3.82257504e-01 -4.82628892e-01 -1.55251502e-01
  -4.66728100e-01 -5.82072121e-01  6.83563961e-02 -1.81819535e-01
  -1.77201438e-01 -3.03542762e-01  1.77830293e+00 -4.51245059e-01
   1.13359812e-01  8.41983267e-01 -4.22262436e-01 -3.01653048e-01
   2.28696957e-01  6.83563961e-02  1.13359812e-01  1.47242305e+00
   1.57678689e+00  1.69095464e+00  1.91396128e+00  9.10121235e-01
   9.01540302e-01  4.43910978e-01  4.94186940e-01  4.25061863e-01
   6.78194083e-01  3.23699837e-01 -2.02358640e-02  1.03749296e+00
   1.36353082e+00  9.75063113e-01  1.27441339e+00  2.19638150e+00
   2.03950032e+00  1.68887112e+00  1.64845411e+00 -7.40970557e-01
  -6.83521842e-01  5.30778344e-01  6.58

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Concatenate, LSTM, Bidirectional, Dense, Lambda, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2

# --- CLEAN THE DATA ---
X_clean = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
y_clean = y

# --- PREPARE INPUTS FOR THE MODEL ---
num_samples = X_clean.shape[0]
num_features_per_fighter = X_clean.shape[2]  # Should be 77
num_referees = int(X_clean[:,:,0].max()) + 1  # Referee is now first feature
num_stances = int(X_clean[:,:,-1].max()) + 1  # Stance is last feature
num_numeric = num_features_per_fighter - 2    # Exclude referee (0) and stance (-1)

# Extract components
referee_input = X_clean[:,:,0].astype(int)[..., np.newaxis]  # Shape: (samples, 2, 1)
stance_input = X_clean[:,:,-1].astype(int)[..., np.newaxis]  # Shape: (samples, 2, 1)
numeric_input = X_clean[:,:,1:-1]  # Shape: (samples, 2, 75)

# --- BUILD ENHANCED MODEL ---
input_referee = Input(shape=(2, 1), name='referee')
input_stance = Input(shape=(2, 1), name='stance')
input_numeric = Input(shape=(2, num_numeric), name='numeric')

# Embedding layers with regularization
emb_referee = Embedding(num_referees, 8, name='referee_embedding', embeddings_regularizer=l2(0.001))(input_referee)
emb_stance = Embedding(num_stances, 4, name='stance_embedding', embeddings_regularizer=l2(0.001))(input_stance)

# Squeeze embedding dimensions
emb_referee = Lambda(lambda x: tf.squeeze(x, axis=2))(emb_referee)
emb_stance = Lambda(lambda x: tf.squeeze(x, axis=2))(emb_stance)

# Concatenate all features
concat = Concatenate(axis=-1)([emb_referee, emb_stance, input_numeric])

# Enhanced BiLSTM architecture
x = Bidirectional(LSTM(64, return_sequences=True))(concat)
x = LayerNormalization()(x)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(32))(x)
x = LayerNormalization()(x)
x = Dropout(0.5)(x)

# Output layer
output = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001))(x)

model = Model(inputs=[input_referee, input_stance, input_numeric], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss='binary_crossentropy',
              metrics=['accuracy', 
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])
model.summary()

# --- TRAIN WITH IMPROVED CALLBACKS ---
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'best_model_enhanced.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

history = model.fit(
    [referee_input, stance_input, numeric_input],
    y_clean,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop, checkpoint],
    class_weight={0: 1, 1: 1.2}  # Adjust if class imbalance exists
)


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ referee             │ (None, 2, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stance (InputLayer) │ (None, 2, 1)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ referee_embedding   │ (None, 2, 1, 8)   │      1,856 │ referee[0][0]     │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stance_embedding    │ (None, 2, 1, 4)   │         24 │ stance[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_12 (Lambda)  │ (None, 2, 8)      │          0 │ referee_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_13 (Lambda)  │ (None, 2, 4)      │          0 │ stance_embedding… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numeric             │ (None, 2, 70)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 2, 82)     │          0 │ lambda_12[0][0],  │
│ (Concatenate)       │                   │            │ lambda_13[0][0],  │
│                     │                   │            │ numeric[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_10    │ (None, 2, 128)    │     75,264 │ concatenate_6[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 2, 128)    │        256 │ bidirectional_10… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 2, 128)    │          0 │ layer_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_11    │ (None, 64)        │     41,216 │ dropout_10[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64)        │        128 │ bidirectional_11… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 64)        │          0 │ layer_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │         65 │ dropout_11[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 118,809 (464.10 KB)

 Trainable params: 118,809 (464.10 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
 57/205 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5222 - loss: 1.0755 - precision: 0.5692 - recall: 0.5880

KeyboardInterrupt: 

In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Concatenate, LSTM, Bidirectional, Dense, Lambda, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2

# Import the first CSV file
# IMPORTANT: Keep 'event_date' for time-based validation
feature_df = pd.read_csv('ufc_features.csv', parse_dates=['event_date'])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Drop the columns (excluding event_date now)
columns_to_drop = ['p1_fighter', 'p2_fighter'] # 'event_date' is kept
feature_df = feature_df.drop(columns=columns_to_drop)
cols_to_drop = [col for col in feature_df.columns if col.startswith('method_')]
feature_df.drop(columns=cols_to_drop, inplace=True)

# Clean all column names
def clean_column_name(col):
  return col.lower().replace(' ', '_').replace('.', '').replace('-', '_')

# Apply to all columns
feature_df.columns = [clean_column_name(col) for col in feature_df.columns]

# Sort by event_date for time-based splitting
feature_df = feature_df.sort_values(by='event_date').reset_index(drop=True)

# --- Encode categorical features ---
# Encode stances
stance_le = LabelEncoder()
all_stances = pd.concat([feature_df['p1_stance'], feature_df['p2_stance']])
stance_le.fit(all_stances)
feature_df['p1_stance'] = stance_le.transform(feature_df['p1_stance'])
feature_df['p2_stance'] = stance_le.transform(feature_df['p2_stance'])

# Encode referee (new additional categorical feature)
referee_le = LabelEncoder()
referee_le.fit(feature_df['referee'])
feature_df['referee'] = referee_le.transform(feature_df['referee'])

# --- Scale numeric features except categorical features ---
numeric_feats = [
    'height', 'weight', 'reach', 'slpm', 'str_acc', 'sapm', 'str_def',
    'td_avg', 'td_acc', 'td_def', 'sub_avg', 'age_at_event',
    # Adding fight history metrics
    # 'days_since_last_fight', 'wins', 'losses', 'total', 'win_streak',
    # Adding age-adjusted metrics
    # 'age_adjusted_slpm', 'age_adjusted_str_acc', 'age_adjusted_sapm',
    # 'age_adjusted_str_def', 'age_adjusted_td_avg', 'age_adjusted_td_acc',
    # 'age_adjusted_td_def', 'age_adjusted_sub_avg',
    # # Adding EMA metrics
    # 'kd_ema', 'sig_str_pct_ema', 'td_pct_ema', 'sub_att_ema', 'rev_ema',
    # 'ctrl_ema', 'r1_kd_ema', 'r1_sig_str_pct_ema', 'r1_td_pct_ema',
    # 'r1_sub_att_ema', 'r1_rev_ema', 'r1_ctrl_ema',
    # 'sig_str_pct_detailed_ema', 'r1_sig_str_pct_detailed_ema',
    # 'sig_str_landed_ema', 'sig_str_attempted_ema',
    # 'total_str_landed_ema', 'total_str_attempted_ema',
    # 'td_landed_ema', 'td_attempted_ema',
    # 'r1_sig_str_landed_ema', 'r1_sig_str_attempted_ema',
    # 'r1_total_str_landed_ema', 'r1_total_str_attempted_ema',
    # 'r1_td_landed_ema', 'r1_td_attempted_ema',
    # 'head_landed_ema', 'head_attempted_ema',
    # 'body_landed_ema', 'body_attempted_ema',
    # 'leg_landed_ema', 'leg_attempted_ema',
    # 'distance_landed_ema', 'distance_attempted_ema',
    # 'clinch_landed_ema', 'clinch_attempted_ema',
    # 'ground_landed_ema', 'ground_attempted_ema',
    # 'r1_head_landed_ema', 'r1_head_attempted_ema',
    # 'r1_body_landed_ema', 'r1_body_attempted_ema',
    # 'r1_leg_landed_ema', 'r1_leg_attempted_ema',
    # 'r1_distance_landed_ema', 'r1_distance_attempted_ema',
    # 'r1_clinch_landed_ema', 'r1_clinch_attempted_ema',
    # 'r1_ground_landed_ema', 'r1_ground_attempted_ema'
]

p1_numeric = ['p1_' + f for f in numeric_feats]
p2_numeric = ['p2_' + f for f in numeric_feats]

# Handle potential NaN values
feature_df[p1_numeric + p2_numeric] = feature_df[p1_numeric + p2_numeric].fillna(0)

scaler = StandardScaler()
feature_df[p1_numeric + p2_numeric] = scaler.fit_transform(feature_df[p1_numeric + p2_numeric])

# --- Prepare features per fighter (including stance) ---
features_per_fighter = numeric_feats + ['stance']
p1_features = ['p1_' + f for f in features_per_fighter]
p2_features = ['p2_' + f for f in features_per_fighter]

# --- Structure data for BiLSTM including referee as a global feature ---
# Create a sequence of [referee, p1_features] and [referee, p2_features] for each sample
X = np.stack([
    np.hstack([feature_df[['referee']].values, feature_df[p1_features].values]),
    np.hstack([feature_df[['referee']].values, feature_df[p2_features].values])
], axis=1)  # shape: (num_samples, 2, 1+num_features_per_fighter)

y = feature_df['winner'].values

# --- Check the shapes and a sample ---
print("X shape:", X.shape)
print("Sample X[0]:\n", X[0])
print("y shape:", y.shape)
print("Sample y:", y[0])
print("Features per fighter (excluding referee):", features_per_fighter)
print("Total features per fighter (including referee):", len(features_per_fighter) + 1)

# --- Time-Based Splitting ---
# Determine the split point (e.g., 80% for training, 20% for validation)
split_point = int(len(feature_df) * 0.8)

# Training data
X_train_referee = X[:split_point, :, 0].astype(int)[..., np.newaxis]
X_train_stance = X[:split_point, :, -1].astype(int)[..., np.newaxis]
X_train_numeric = X[:split_point, :, 1:-1]
y_train = y[:split_point]

# Validation data
X_val_referee = X[split_point:, :, 0].astype(int)[..., np.newaxis]
X_val_stance = X[split_point:, :, -1].astype(int)[..., np.newaxis]
X_val_numeric = X[split_point:, :, 1:-1]
y_val = y[split_point:]

print("\n--- Time-Based Split Shapes ---")
print("X_train_referee shape:", X_train_referee.shape)
print("X_train_stance shape:", X_train_stance.shape)
print("X_train_numeric shape:", X_train_numeric.shape)
print("y_train shape:", y_train.shape)
print("X_val_referee shape:", X_val_referee.shape)
print("X_val_stance shape:", X_val_stance.shape)
print("X_val_numeric shape:", X_val_numeric.shape)
print("y_val shape:", y_val.shape)

# --- CLEAN THE DATA (already done for X, just ensure no NaNs in Y) ---
y_train_clean = y_train
y_val_clean = y_val

# --- PREPARE INPUTS FOR THE MODEL ---
# These are already prepared in the time-based splitting section
num_features_per_fighter = X.shape[2]
num_referees = int(X[:,:,0].max()) + 1
num_stances = int(X[:,:,-1].max()) + 1
num_numeric = num_features_per_fighter - 2

# --- BUILD ENHANCED MODEL (remains the same) ---
input_referee = Input(shape=(2, 1), name='referee')
input_stance = Input(shape=(2, 1), name='stance')
input_numeric = Input(shape=(2, num_numeric), name='numeric')

# Embedding layers with regularization
emb_referee = Embedding(num_referees, 8, name='referee_embedding', embeddings_regularizer=l2(0.001))(input_referee)
emb_stance = Embedding(num_stances, 4, name='stance_embedding', embeddings_regularizer=l2(0.001))(input_stance)

# Squeeze embedding dimensions
emb_referee = Lambda(lambda x: tf.squeeze(x, axis=2))(emb_referee)
emb_stance = Lambda(lambda x: tf.squeeze(x, axis=2))(emb_stance)

# Concatenate all features
concat = Concatenate(axis=-1)([emb_referee, emb_stance, input_numeric])

# Enhanced BiLSTM architecture
x = Bidirectional(LSTM(64, return_sequences=True))(concat)
x = LayerNormalization()(x)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(32))(x)
x = LayerNormalization()(x)
x = Dropout(0.5)(x)

# Output layer
output = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001))(x)

model = Model(inputs=[input_referee, input_stance, input_numeric], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])
model.summary()

# --- TRAIN WITH IMPROVED CALLBACKS ---
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'best_model_enhanced.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

history = model.fit(
    [X_train_referee, X_train_stance, X_train_numeric],
    y_train_clean,
    epochs=50,
    batch_size=32,
    validation_data=([X_val_referee, X_val_stance, X_val_numeric], y_val_clean), # Use time-based validation data
    callbacks=[early_stop, checkpoint],
    class_weight={0: 1, 1: 1.2}  # Adjust if class imbalance exists
)

X shape: (8176, 2, 14)
Sample X[0]:
 [[ 1.00000000e+02 -5.47078056e-02  1.16358820e+00 -4.21557461e+00
  -2.63418383e+00 -4.96570215e+00 -2.73199071e+00 -5.85799874e+00
  -1.25596182e+00 -2.08926330e+00 -3.02186878e+00 -8.46211610e-01
  -6.11194668e+00  1.00000000e+00]
 [ 1.00000000e+02  4.13889687e-01  2.07651219e-01 -2.81305510e+00
  -2.31454391e+00 -4.06847179e+00 -2.38358504e+00 -4.72717899e+00
  -1.13836282e+00 -1.71133563e+00 -2.42421059e+00 -7.44299746e-01
  -1.93478120e+00  5.00000000e+00]]
y shape: (8176,)
Sample y: 1
Features per fighter (excluding referee): ['height', 'weight', 'reach', 'slpm', 'str_acc', 'sapm', 'str_def', 'td_avg', 'td_acc', 'td_def', 'sub_avg', 'age_at_event', 'stance']
Total features per fighter (including referee): 14

--- Time-Based Split Shapes ---
X_train_referee shape: (6540, 2, 1)
X_train_stance shape: (6540, 2, 1)
X_train_numeric shape: (6540, 2, 12)
y_train shape: (6540,)
X_val_referee shape: (1636, 2, 1)
X_val_stance shape: (1636, 2, 1)
X_val_nu

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ referee             │ (None, 2, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stance (InputLayer) │ (None, 2, 1)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ referee_embedding   │ (None, 2, 1, 8)   │      1,856 │ referee[0][0]     │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stance_embedding    │ (None, 2, 1, 4)   │         24 │ stance[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_18 (Lambda)  │ (None, 2, 8)      │          0 │ referee_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_19 (Lambda)  │ (None, 2, 4)      │          0 │ stance_embedding… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numeric             │ (None, 2, 12)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_9       │ (None, 2, 24)     │          0 │ lambda_18[0][0],  │
│ (Concatenate)       │                   │            │ lambda_19[0][0],  │
│                     │                   │            │ numeric[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_16    │ (None, 2, 128)    │     45,568 │ concatenate_9[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 2, 128)    │        256 │ bidirectional_16… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 2, 128)    │          0 │ layer_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_17    │ (None, 64)        │     41,216 │ dropout_16[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64)        │        128 │ bidirectional_17… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 64)        │          0 │ layer_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1)         │         65 │ dropout_17[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 89,113 (348.10 KB)

 Trainable params: 89,113 (348.10 KB)

...

205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6549 - loss: 0.6995 - precision: 0.7017 - recall: 0.8261 - val_accuracy: 0.6583 - val_loss: 0.6211 - val_precision: 0.6497 - val_recall: 0.8399
Epoch 5/50
204/205 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6649 - loss: 0.6674 - precision: 0.7048 - recall: 0.8425

205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6650 - loss: 0.6673 - precision: 0.7049 - recall: 0.8425 - val_accuracy: 0.6595 - val_loss: 0.6188 - val_precision: 0.6433 - val_recall: 0.8739
Epoch 6/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6759 - loss: 0.6555 - precision: 0.7053 - recall: 0.8651 - val_accuracy: 0.6583 - val_loss: 0.6136 - val_precision: 0.6507 - val_recall: 0.8355
Epoch 7/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6891 - loss: 0.6297 - precision: 0.7182 - recall: 0.8723 - val_accuracy: 0.6479 - val_loss: 0.6180 - val_precision: 0.6342 - val_recall: 0.8706
Epoch 8/50
195/205 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6839 - loss: 0.6319 - precision: 0.7115 - recall: 0.8743

205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6838 - loss: 0.6318 - precision: 0.7114 - recall: 0.8741 - val_accuracy: 0.6656 - val_loss: 0.6178 - val_precision: 0.6594 - val_recall: 0.8279
Epoch 9/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6887 - loss: 0.6233 - precision: 0.7134 - recall: 0.8753 - val_accuracy: 0.6638 - val_loss: 0.6192 - val_precision: 0.6508 - val_recall: 0.8564
Epoch 10/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6980 - loss: 0.5973 - precision: 0.7184 - recall: 0.8829 - val_accuracy: 0.6546 - val_loss: 0.6194 - val_precision: 0.6394 - val_recall: 0.8728
Epoch 11/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7014 - loss: 0.5994 - precision: 0.7193 - recall: 0.8943 - val_accuracy: 0.6650 - val_loss: 0.6184 - val_precision: 0.6504 - val_recall: 0.8629
Epoch 12/50
194/205 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7095 - loss: 0.5959 - precision: 0.7279 - recall: 0.8991

205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7091 - loss: 0.5962 - precision: 0.7275 - recall: 0.8986 - val_accuracy: 0.6681 - val_loss: 0.6159 - val_precision: 0.6614 - val_recall: 0.8289
Epoch 13/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7007 - loss: 0.5978 - precision: 0.7201 - recall: 0.8903

205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7007 - loss: 0.5978 - precision: 0.7201 - recall: 0.8903 - val_accuracy: 0.6754 - val_loss: 0.6134 - val_precision: 0.6715 - val_recall: 0.8180
Epoch 14/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7176 - loss: 0.5742 - precision: 0.7403 - recall: 0.8885 - val_accuracy: 0.6656 - val_loss: 0.6162 - val_precision: 0.6594 - val_recall: 0.8279
Epoch 15/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7306 - loss: 0.5765 - precision: 0.7438 - recall: 0.8974 - val_accuracy: 0.6583 - val_loss: 0.6213 - val_precision: 0.6465 - val_recall: 0.8542
Epoch 16/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7235 - loss: 0.5745 - precision: 0.7345 - recall: 0.9139 - val_accuracy: 0.6620 - val_loss: 0.6180 - val_precision: 0.6584 - val_recall: 0.8180
